In [79]:
from browser_libs import get_collection_items, get_collection_names, get_collection_for_user
from memoize import memoize # pip install memoize2
from collections import Counter

In [80]:
@memoize
def get_users_with_choose_difficulty():
  collection_names = get_collection_names()
  output = []
  for x in collection_names:
    if x.endswith('_internal:choose_difficulty'):
      user = x.replace('_internal:choose_difficulty', '')
      output.append(user)
  return output


In [81]:
def get_choose_difficulty_items_for_user(user):
  collection_items = get_collection_for_user(user, 'internal:choose_difficulty')
  output = []
  for x in collection_items:
    if 'is_new_session' not in x:
      continue
    if x['is_new_session'] != True:
      continue
    if ('developer_mode' in x) and (x['developer_mode'] == True):
      continue
    if ('is_preview_mode' in x) and (x['is_preview_mode'] == True):
      continue
    if ('is_suggestion_mode' in x) and (x['is_suggestion_mode'] == True):
      continue
    output.append(x)
  return output

In [82]:
def get_choose_difficulty_counts_for_user(user):
  collection_items = get_choose_difficulty_items_for_user(user)
  output = {}
  for x in collection_items:
    if 'type' not in x:
      continue
    if x['type'] != 'action':
      continue
    if 'difficulty' not in x:
      continue
    difficulty = x['difficulty']
    if difficulty not in output:
      output[difficulty] = 1
    else:
      output[difficulty] += 1
  return output

In [83]:
def get_choose_difficulty_counts_for_all_users():
  output = Counter()
  for user in get_users_with_choose_difficulty():
    difficulty_counts = get_choose_difficulty_counts_for_user(user)
    for difficulty,count in difficulty_counts.items():
      output[difficulty] += count
  return output

In [84]:
def get_total_counts_for_user(user):
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  return sum(difficulty_counts.values())

In [85]:
def num_types_tried(user):
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  return len(difficulty_counts.keys())

In [86]:
def did_user_try_multiple(user):
  return num_types_tried(user) > 1

In [87]:
def get_try_multiple_counts_for_all_users():
  output = {
    'none_total': 0,
    'one_total': 0,
    'one_tried': 0,
    'multiple_tried': 0,
  }
  for user in get_users_with_choose_difficulty():
    total_counts = get_total_counts_for_user(user)
    if total_counts == 0:
      output['none_total'] += 1
      continue
    if total_counts == 1:
      output['one_total'] += 1
      continue
    if did_user_try_multiple(user):
      output['multiple_tried'] += 1
    else:
      output['one_tried'] += 1
  return output

In [88]:
def get_breakdown_for_one_tried():
  output = Counter()
  for user in get_users_with_choose_difficulty():
    total_counts = get_total_counts_for_user(user)
    if total_counts <= 1:
      continue
    if did_user_try_multiple(user):
      continue
    difficulty_counts = get_choose_difficulty_counts_for_user(user)
    chosen_difficulty = list(difficulty_counts.keys())[0]
    output[chosen_difficulty] += difficulty_counts[chosen_difficulty]
  return output

In [89]:
print(len(get_users_with_choose_difficulty()))

279


In [90]:
print(get_choose_difficulty_counts_for_all_users())

Counter({'nothing': 5837, 'easy': 1607, 'medium': 1239, 'hard': 460})


In [91]:
print(get_try_multiple_counts_for_all_users())

{'none_total': 11, 'one_total': 57, 'one_tried': 73, 'multiple_tried': 138}


In [92]:
print(get_breakdown_for_one_tried())

Counter({'nothing': 601, 'easy': 280, 'medium': 238})


In [93]:
print(get_choose_difficulty_counts_for_user('f4d95006c663a799d9185576'))

{'medium': 1, 'hard': 1}


In [94]:
#print(collection_names[0])
difficulty_items = get_choose_difficulty_items_for_user('f4d95006c663a799d9185576')
for x in difficulty_items:
  print(x)

{'_id': ObjectId('5c101ac9c6466e0014dba890'), 'url': 'https://www.youtube.com/', 'tab_id': 1456145845, 'session_id': 0, 'is_new_session': True, 'is_preview_mode': False, 'is_suggestion_mode': False, 'is_previously_seen': False, 'type': 'impression', 'intervention': 'internal/choose_difficulty', 'userid': 'f4d95006c663a799d9185576', 'install_id': '8132291a2f7334b45cfebbba', 'day': 1075, 'synced': 0, 'timestamp': 1544559305512.0, 'localtime': 'Tue Dec 11 2018 12:15:04 GMT-0800 (北美太平洋标准时间)', 'itemid': '2aa218b1ccde7f3e3782d08e', 'log_major_ver': '8', 'log_minor_ver': '1', 'habitlab_version': '1.0.257', 'unofficial_version': 'bleifeoekkfhicamkpadfoclfhfmmina', 'id': 1, 'logname': 'internal/choose_difficulty', 'timestamp_local': 1544559304784.0}
{'_id': ObjectId('5c101acb7a96610014559a49'), 'difficulty': 'medium', 'intervention_name': 'youtube/remove_recommended', 'new_session': True, 'url': 'https://www.youtube.com/', 'tab_id': 1456145845, 'session_id': 0, 'is_new_session': True, 'is_previ

In [95]:
#choose_difficulty_set = set()
#for x in collection_names:
#  if 'difficulty' in x:
#    choose_difficulty_set.add(x)

In [96]:
#print(choose_difficulty_set)
#for x in choose_difficulty_set:
  